# EEG Classification - PyTorch
updated: Sep. 01, 2018

Data: https://www.physionet.org/pn4/eegmmidb/

In [1]:
# System
import requests
import re
import os
from io import StringIO
import shutil
import pathlib
import urllib

from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

# Essential Data Handling
import numpy as np
import pandas as pd
from math import ceil, floor

# Get Paths
from glob import glob

# EEG package
from mne import pick_types, events_from_annotations
from mne.io import read_raw_edf

import pickle
import sys
import json

from datetime import datetime

# PyThorch 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset

# To use Tensorflow with PyTorch
from torchbearer import Trial
from torchbearer.callbacks import TensorBoard, Best, ReduceLROnPlateau, CSVLogger, ModelCheckpoint

# To parse input arguments
import argparse

## Data Description

Subjects performed different motor/imagery tasks while 64-channel EEG were recorded using the BCI2000 system (http://www.bci2000.org). Each subject performed 14 experimental runs: 

- two one-minute baseline runs (one with eyes open, one with eyes closed)
- three two-minute runs of each of the four following tasks:
    - 1:
        - A target appears on either the left or the right side of the screen. 
        - The subject opens and closes the corresponding fist until the target disappears. 
        - Then the subject relaxes.
    - 2:
        - A target appears on either the left or the right side of the screen. 
        - The subject imagines opening and closing the corresponding fist until the target disappears. 
        - Then the subject relaxes.
    - 3:
        - A target appears on either the top or the bottom of the screen. 
        - The subject opens and closes either both fists (if the target is on top) or both feet (if the target is on the bottom) until the target disappears. 
        - Then the subject relaxes.
    - 4:
        - A target appears on either the top or the bottom of the screen. 
        - The subject imagines opening and closing either both fists (if the target is on top) or both feet (if the target is on the bottom) until the target disappears. 
        - Then the subject relaxes.

The data are provided here in EDF+ format (containing 64 EEG signals, each sampled at 160 samples per second, and an annotation channel). 
For use with PhysioToolkit software, rdedfann generated a separate PhysioBank-compatible annotation file (with the suffix .event) for each recording. 
The .event files and the annotation channels in the corresponding .edf files contain identical data.

# Summary tasks

Remembering that:

    - Task 1 (open and close left or right fist)
    - Task 2 (imagine opening and closing left or right fist)
    - Task 3 (open and close both fists or both feet)
    - Task 4 (imagine opening and closing both fists or both feet)

we will referred to 'Task *' with the meneaning above. 

In summary, the experimental runs were:

1.  Baseline, eyes open
2.  Baseline, eyes closed
3.  Task 1 
4.  Task -2 
5.  Task --3 
6.  Task ---4 
7.  Task 1
8.  Task -2
9.  Task --3
10. Task ---4
11. Task 1
12. Task -2
13. Task --3
14. Task ---4

# Annotation

Each annotation includes one of three codes (T0, T1, or T2):

- T0 corresponds to rest
- T1 corresponds to onset of motion (real or imagined) of
    - the left fist (in runs 3, 4, 7, 8, 11, and 12)
    - both fists (in runs 5, 6, 9, 10, 13, and 14)
- T2 corresponds to onset of motion (real or imagined) of
    - the right fist (in runs 3, 4, 7, 8, 11, and 12)
    - both feet (in runs 5, 6, 9, 10, 13, and 14)
    
In the BCI2000-format versions of these files, which may be available from the contributors of this data set, these annotations are encoded as values of 0, 1, or 2 in the TargetCode state variable.

{'T0':0, 'T1':1, 'T2':2}

In our experiments we will see only :

- run_type_0:
    - append_X
- run_type_1
    - append_X_y
- run_type_2
    - append_X_y
    
and the coding is: 

- T0 corresponds to rest 
    - (2)
- T1 (real or imagined)
    - (4,  8, 12) the left fist 
    - (6, 10, 14) both fists 
- T2 (real or imagined)
    - (4,  8, 12) the right fist 
    - (6, 10, 14) both feet 

## 4. Modeling - Time-Distributed CNN + RNN

Training Plan:

+ 4 GPU units (Nvidia Tesla P100) were used to train this neural network.
+ Instead of training the whole model at once, I trained the first block (CNN) first. Then using the trained parameters as initial values, I trained the next blocks step-by-step. This approach can greatly reduce the time required for training and help avoiding falling into local minimums.
+ The first blocks (CNN) can be applied for other EEG classification models as a pre-trained base.

+ The initial learning rate is set to be $10^{3}$ with Adam optimization. I used several callbacks such as ReduceLROnPlateau which adjusts the learning rate at local minima. Also, I record the log for tensorboard to monitor the training process.

### 4.1 Pytorch Implementation

Based on MNIST CNN + LSTM example

In [2]:
# Network Parameters
# num_input = X_train.shape[0]   # PhysioNet data input (mesh shape: 10*11)

class Args:
    def __init__(self):
        self.test_rate  = 0.2
        self.run_number = 0
        self.cuda       = False
        self.no_cuda    = True
        self.seed       = 42
        self.batch_size = 128
        self.val_batch_size  = 128
        self.test_batch_size = 1000
        self.epochs         = 10
        self.lr             = 0.001
        self.momentum       = 0.5
        self.log_interval   = 10
        self.n_nodes        = [3,2,2]# . structure of network (e.s. [3,2,2]: 3 CNN + 2 LSTM + 2 FC )
        self.cnn_filter     = 32     # . number filter for the first layer of CNN
        self.cnn_kernelsize = 3      # . dimension of kernel for CNN layer
        self.lstm_hidden    = 64     # . number of elements for hidden state of lstm
        self.fc_dim         = 1024   # . number of neurons for fully connected layer
        self.fc_dropout     = 0.5    # . drop out rate for fully connected layer
        self.rcnn_output    = 5      # . output of network
        self.num_classes    = 5      # PhysioNet total classes
        self.split_type = 'user_dependent' 
     

args = Args()

args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

In [4]:
[X_train, y_train, p_train] = pickle.load( open( "./dataset/splitted_data/"+args.split_type+"/test_rate_"+str(args.test_rate)+"/seed_"+str(args.seed)+"/train.p", "rb" ) )
[X_val, y_val, p_val]       = pickle.load( open( "./dataset/splitted_data/"+args.split_type+"/test_rate_"+str(args.test_rate)+"/seed_"+str(args.seed)+"/val.p"  , "rb" ) )
[X_test, y_test, p_test]    = pickle.load( open( "./dataset/splitted_data/"+args.split_type+"/test_rate_"+str(args.test_rate)+"/seed_"+str(args.seed)+"/test.p" , "rb" ) )

In [5]:
print('train', 'X', X_train.shape, 'y', y_train.shape) #, 'p', p_train.shape)
print('val  ', 'X', X_val.shape  , 'y', y_val.shape) #, 'p', p_val.shape)
print('test ', 'X', X_test.shape , 'y', y_test.shape) #, 'p', p_test.shape)

train X (316593, 10, 10, 11, 1) y (316593, 1)
val   X (79149, 10, 10, 11, 1) y (79149, 1)
test  X (123670, 10, 10, 11, 1) y (123670, 1)


In [32]:
print('\nsqueeze of all X, y - train, val, test\n')
X_train = X_train.squeeze()
y_train = y_train.squeeze()
X_val = X_val.squeeze()
y_val = y_val.squeeze()
X_test = X_test.squeeze()
y_test = y_test.squeeze()

print('train', 'X', X_train.shape, 'y', y_train.shape) #, 'p', p_train.shape)
print('val  ', 'X', X_val.shape  , 'y', y_val.shape) #, 'p', p_val.shape)
print('test ', 'X', X_test.shape , 'y', y_test.shape) #, 'p', p_test.shape)


squeeze of all X, y - train, val, test

train X (316593, 10, 10, 11) y (316593,)
val   X (79149, 10, 10, 11) y (79149,)
test  X (123670, 10, 10, 11) y (123670,)


In [33]:
class EEGImagesDatasetLoader(Dataset):
    """EEGs (converted in images) dataset."""

    def __init__(self, pickle_file='train.p', root_dir='./py/stack/', transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        [self.X, self.y] = pickle.load( open( root_dir + pickle_file, "rb" ) )
        
        '''
        def one_hot_embedding(labels, num_classes=args.num_classes):
            """Embedding labels to one-hot form.

            Args:
              labels: (LongTensor) class labels, sized [N,].
              num_classes: (int) number of classes.

            Returns:
              (tensor) encoded labels, sized [N, #classes].
            """
            y = torch.eye(num_classes) 
            return y[labels] 
        '''

        # self.y = one_hot_embedding(self.y, args.num_classes )
        # print(self.y)
        self.y = torch.tensor(self.y, dtype=torch.long)
        # print(self.y.size())
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]

        if self.transform:
            # If the transform variable is not empty
            # then it applies the operations in the transforms with the order that it is created.
            image = self.transform(image)

        return (image, label)

In [34]:
class EEGImagesDatasetHolding(Dataset):
    """EEGs (converted in images) dataset."""

    def __init__(self, X, y, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.X, self.y = X, y
        
        '''
        def one_hot_embedding(labels, num_classes=args.num_classes):
            """Embedding labels to one-hot form.

            Args:
              labels: (LongTensor) class labels, sized [N,].
              num_classes: (int) number of classes.

            Returns:
              (tensor) encoded labels, sized [N, #classes].
            """
            y = torch.eye(num_classes) 
            return y[labels] 
        '''

        # self.y = one_hot_embedding(self.y, args.num_classes )
        # print(self.y)
        self.y = torch.tensor(self.y, dtype=torch.long)
        # print(self.y.size())
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]

        if self.transform:
            # If the transform variable is not empty
            # then it applies the operations in the transforms with the order that it is created.
            image = self.transform(image)

        return (image, label)

In [46]:
eeg_train = EEGImagesDatasetHolding(X_train, y_train, transform=None)
eeg_val   = EEGImagesDatasetHolding(X_val  , y_val  , transform=None)
eeg_test  = EEGImagesDatasetHolding(X_test , y_test , transform=None)

In [47]:
train_loader = torch.utils.data.DataLoader(
    eeg_train,
    batch_size=args.batch_size,
    shuffle=True,
    **kwargs)

val_loader = torch.utils.data.DataLoader(
    eeg_val,
    batch_size=args.val_batch_size,
    shuffle=True,
    **kwargs)

test_loader = torch.utils.data.DataLoader(
    eeg_test,
    batch_size=args.test_batch_size,
    shuffle=True,
    **kwargs)

In [48]:
(image, label) = eeg_train[0]
print('train dimension:',eeg_train.__len__(), image.shape, label.shape)

(image, label) = eeg_val[0]
print('val   dimension:',eeg_val.__len__(), image.shape, label.shape)

(image, label) = eeg_test[0]
print('test  dimension:',eeg_test.__len__(), image.shape, label.shape)

train dimension: 316593 (10, 10, 11) torch.Size([])
train dimension: 79149 (10, 10, 11) torch.Size([])
test  dimension: 123670 (10, 10, 11) torch.Size([])


In [49]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1  = nn.Conv2d(10, args.cnn_filter, kernel_size=args.cnn_kernelsize)
        self.batch1 = nn.BatchNorm2d(args.cnn_filter)
        # self.act1   = F.elu(args.cnn_filter, alpha=1.0, inplace=False)
        
        self.conv2 = nn.Conv2d(args.cnn_filter*1, args.cnn_filter*2, kernel_size=args.cnn_kernelsize)
        self.batch2 = nn.BatchNorm2d(args.cnn_filter*2**1)
        # self.act2   = F.elu(args.cnn_filter*2**1, alpha=1.0, inplace=False)
        
        self.conv3 = nn.Conv2d( args.cnn_filter * 2, args.cnn_filter * 2**2, kernel_size=args.cnn_kernelsize)
        self.batch3 = nn.BatchNorm2d(args.cnn_filter*2**2)
        # self.act3   = F.elu(args.cnn_filter*2**2, alpha=1.0, inplace=False)
        
        self.FC1 = nn.Linear(2560, args.fc_dim)
        self.dropout1 = nn.Dropout2d(p=args.fc_dropout)
        self.batch4 = nn.BatchNorm1d(args.fc_dim)

    def forward(self, x):
        # print('cnn i', x.shape)
        x = F.elu(self.batch1(self.conv1(x)))
        # print('cnn 1', x.shape)
        x = F.elu(self.batch2(self.conv2(x)))
        # print('cnn 2', x.shape)
        x = F.elu(self.batch3(self.conv3(x)))
        # print('cnn 3', x.shape)
        x = x.view(-1, 2560) # 2560 = args.batch_size * x.shape[-1] * x.shape[-2]
        # print('cnn o/fc1 in', x.shape)
        x = F.elu(self.batch4(self.dropout1(self.FC1(x))))
        # print('fc1 o', x.shape)
        return x

In [50]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        self.cnn = CNN() 
        self.rnn = nn.LSTM(
                input_size  = args.fc_dim, 
                hidden_size = args.lstm_hidden, 
                num_layers  = 2, 
                batch_first = True )
        self.FC2 = nn.Linear(args.lstm_hidden, args.fc_dim)
        self.dropout2 = nn.Dropout(p=args.fc_dropout)
        self.FC3 = nn.Linear(args.fc_dim, args.num_classes)

    def forward(self, x):
        # print('combine i', x.size())
        batch_size, C, H, W = x.size()
        c_in = x.view(batch_size , C, H, W)
        # print('combine cin', c_in.size())
        c_out = self.cnn(c_in)
        # print('combine cout/rin', c_out.size())
        r_in = c_out.view(batch_size, 1, -1)
        # print('combine rin', r_in.size())
        r_out, (h_n, h_c) = self.rnn(r_in)
        # print('combine rout', r_out.size(), '(h_n, h_c)', (h_n.size(), h_c.size()))
        r_out2 = self.FC3(self.dropout2(self.FC2(r_out[:, -1, :])))
        # print('combine o', r_out2.size())
        return r_out2 # F.log_softmax(r_out2, dim=1)

In [51]:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
run_to_path = str(args.run_number)+str(args.n_nodes)+'_'+str(current_time)

my_log = './torchbearer/log_dir/'
my_log_dir = my_log +'CNN_LSTM_run'+run_to_path+'/'
if os.path.exists(os.path.dirname(my_log_dir)):
    for file in os.listdir(my_log_dir):
        file_path = os.path.join(my_log_dir, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(e)
else:
    os.makedirs(os.path.dirname(my_log_dir))

save_model_dir = './torchbearer/save_model/CNN_LSTM_run'+run_to_path+'/'
if os.path.exists(os.path.dirname(save_model_dir)):
    for file in os.listdir(save_model_dir):
        file_path = os.path.join(save_model_dir, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(e)
else:
    os.makedirs(os.path.dirname(save_model_dir))

In [52]:
def write_report(model, history):
    infopath = save_model_dir + "/info.txt"
    with open(infopath, 'w') as fh:
        fh.write("Training parameters : \n")
        fh.write("Input dimensions (train) :  X " + str(X_train.shape) + ", y " + str(X_train.shape) + "\n")
        fh.write("Input dimensions (val  ) :  X " + str(X_val.shape)   + ", y " + str(X_val.shape)   + "\n")
        fh.write("Input dimensions (test ) :  X " + str(X_test.shape)  + ", y " + str(X_test.shape)  + "\n")
        fh.write("Epochs - LR - Dropout - Momentum : " + str(args.epochs) + " - " + str(args.lr) + " - " + str(args.fc_dropout) + "-" + str(args.momentum) + "\n")
        fh.write("Batch_size - Steps_train - Steps_valid : " + str(args.batch_size) + " - " + str(args.val_batch_size) + " - " + str(args.test_batch_size) +"\n")
        fh.write("Final loss - val_loss : " + str(min([ history[i]['loss'] for i in range(len(history))])) + " - " + str(min([ history[i]['val_loss'] for i in range(len(history))])) + "\n")
        fh.write("Network architecture : \n")
        # string = model.state_dict() 
        # Pass the file handle in as a lambda function to make it callable
        # model.summary(print_fn=lambda x: fh.write(x + '\n'))
        print( model, file=fh)
        

callbacks_list = [
                # Best(save_model_dir + 'model_EP{epoch:02d}_VA{val_acc:.4f}.pt', save_model_params_only=True), # monitor='val_acc', mode='max'),
                # ExponentialLR(gamma=0.1),
                # TensorBoardText(comment=current_time),
                ModelCheckpoint(save_model_dir + 'model_[epo]{epoch:02d}_[val]{val_acc:.4f}.pt', save_best_only=True, monitor='val_loss'),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
                CSVLogger(my_log_dir + "log.csv", separator=',', append=True),
                TensorBoard(my_log, write_graph=True, write_batch_metrics=False, write_epoch_metrics=False, comment='run'+run_to_path),
                TensorBoard(my_log, write_graph=False, write_batch_metrics=True, batch_step_size=10, write_epoch_metrics=False, comment='run'+run_to_path),
                TensorBoard(my_log, write_graph=False, write_batch_metrics=False, write_epoch_metrics=True, comment='run'+run_to_path)]


In [53]:
model = CNN_LSTM()

In [54]:
if args.cuda:
    model.cuda()
    
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
loss = nn.CrossEntropyLoss()

In [55]:
trial = Trial(model, optimizer, loss, metrics=['acc', 'loss'], callbacks=callbacks_list) #.to('cuda')
history = trial.with_generators(train_generator=train_loader, val_generator=val_loader, test_generator=test_loader, train_steps=79, val_steps=8, test_steps=8).run(args.epochs)
print(history)

[{'running_acc': 0.21046875417232513, 'running_loss': 1.6096813678741455, 'acc': 0.20856408774852753, 'loss': 1.6097244024276733, 'val_acc': 0.216796875, 'val_loss': 1.6094927787780762, 'train_steps': 79, 'validation_steps': 8}]


In [45]:
old_stdout = sys.stdout
sys.stdout = mystdout = StringIO()
os.system('cat /proc/sys/kernel/hostname')
sys.stdout = old_stdout

d = '######################           '+mystdout.getvalue()
d = '######################           Network History'
for i, epoch in enumerate(history):
    d += '\n ####    EPOCH '+str(i)
    for k, v in epoch.items():
        d += '\n'+str(k)+' : '+str(v)
    d += '\n\n'
print(d)
os.system("curl -X POST -H 'Content-type: application/json' --data '{\"text\":\" "+d+"\"}' https://hooks.slack.com/services/mysercret")

write_report(model, history)

######################           Network History
 ####    EPOCH 0
running_acc : 0.20125000178813934
running_loss : 1.6104434728622437
acc : 0.19740000367164612
loss : 1.6109141111373901
val_acc : 0.21199999749660492
val_loss : 1.6085139513015747
train_steps : 79
validation_steps : 8


